In [1]:
#FinalBob.ipynb
#Noah Walsh, Ben Valois, Jake Hamilton, Derek Windahl
#Miniproject 2. In this project, we use an RNN trained on 
#Spongebob scripts to generate a Spongebob episode
#NLTK might need to be installed via anaconda prompt 
#10/25/2018

# imports needed
import numpy
import sys
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
#nltk used for natural language processing
import nltk
nltk.download('punkt')

Using TensorFlow backend.


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\valoi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
#load our file and convert to a consistent case
#make sure file is in the same directory as the notebook
filename = "Spng.txt"
raw_text = open(filename).read()
#set text to lower case
raw_text = raw_text.lower()
#remove all punctuation
for char in ["[","]",".","?","'","#","\n",",","!","@","$","%","^","&","*","(",")","+","-","_",";",":","<",">",'"',"{","}","\\","/","~","`"]:
    if char in raw_text:
        raw_text = raw_text.replace(char,"")
#tokenize words           
proc_text = nltk.word_tokenize(raw_text)


In [3]:
#map our words to integers so that we can use them properly
#chars = sorted(list(set(raw_text)))
#char_to_int = dict((c, i) for i, c in enumerate(chars))

words = sorted(list(set(proc_text)))
word_to_int = dict((w, i) for i, w in enumerate(words))


In [4]:
#split our text into our X and Y vectors
#n_chars = len(raw_text)
#n_vocab = len(chars)

myWords = len(proc_text)
myVocab = len(words)
#number of words per sequence
seq_length = 20
dataX = []
dataY = []
#creates sequences/patterns
for i in range(0, myWords - seq_length, 1):
  seq_in = proc_text[i:i + seq_length]
  seq_out = proc_text[i + seq_length]
  dataX.append([word_to_int[word] for word in seq_in])
  dataY.append(word_to_int[seq_out])
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)

Total Patterns:  10617


In [5]:
#work with the resulting data to make sure that it's in a form that keras will take
X = numpy.reshape(dataX, (n_patterns, seq_length))
#X = X / float(n_vocab)
y = np_utils.to_categorical(dataY)
X.shape[0]

10617

In [6]:
#create the model
#RNN hidden dimension
hidden_size = 800
num_steps = seq_length

model = Sequential()
#perform word embedding 
model.add(Embedding(myVocab, hidden_size, input_length = num_steps))
#long short term memory layer (LSTM layer)
model.add(LSTM(hidden_size, input_shape=(X.shape), return_sequences=True))
#dropout to improve generated text
model.add(Dropout(0.5))
#2nd LSTM layer
model.add(LSTM(hidden_size))
model.add(Dropout(0.5))
#final normalized layer
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
#model fit with 57 epochs found to be ideal number
model.fit(X, y, epochs=57, batch_size=128)

Epoch 1/57
10617/10617 [==============================] - 30s 3ms/step - loss: 6.5278: 0s - loss: 6.53
Epoch 2/57
10617/10617 [==============================] - 8s 743us/step - loss: 6.2293 0s - los
Epoch 3/57
10617/10617 [==============================] - 8s 743us/step - loss: 6.1679
Epoch 4/57
10617/10617 [==============================] - 8s 746us/step - loss: 6.0060
Epoch 5/57
10617/10617 [==============================] - 8s 741us/step - loss: 5.9082
Epoch 6/57
10617/10617 [==============================] - 8s 743us/step - loss: 5.8199
Epoch 7/57
10617/10617 [==============================] - 8s 742us/step - loss: 5.7220
Epoch 8/57
10617/10617 [==============================] - 8s 741us/step - loss: 5.6321
Epoch 9/57
10617/10617 [==============================] - 8s 740us/step - loss: 5.5272
Epoch 10/57
10617/10617 [==============================] - 8s 741us/step - loss: 5.4173
Epoch 11/57
10617/10617 [==============================] - 8s 744us/step - loss: 5.2933
Epoch 12/57
1061

In [7]:
#create our prediction
#get word from integer
int_to_word = dict((i, w) for i, w in enumerate(words))
oot = ""
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("Seed:")
print("\"", ''.join([(int_to_word[value]+" ") for value in pattern]), "\"")
# generates 500 words
for i in range(500):
  x = numpy.reshape(pattern, (1, len(pattern)))
  #x = x / float(n_vocab)
  prediction = model.predict(x, verbose=0)
  index = numpy.argmax(prediction)
  result = int_to_word[index]
  seq_in = [int_to_word[value] for value in pattern]
  oot = oot +" "+ result
  pattern.append(index)
  pattern = pattern[1:len(pattern)]
print("\nDone.")
print(oot)

Seed:
" by the power invested in me i now pronounce you man and wife manager enters the cafeteria what is going  "

Done.
 on in here mermaid man points to spongebob who stops smiling you may kiss the bride spongebob gets kicked out and rolls home patrick did you reunite our heroes spongebob no but im married back at the retirement home mermaid man he and barnacle boy are in rocking chairs up up and away up up and away spongebob dressed up as a woman and speaking in a southern accent oh mah this purse is so big and heavy patrick dressed up as a robber hold it right there maam ill be taking that grabs spongebobs purse spongebob screams haylp haylp haylp patrick its working spongebob barnacle boy comes over to him wha are you here to rescue little ol me barnacle boy yells at spongebob ruining his makeup pipe down you could wake mermaid man and hes ornery when his nap is disturbed spongebob approaches mermaid man who is sleeping with his eyes open ever alert mermaid man has trained himsel